# Import the Dependencies

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load the Dataset

In [23]:
df = pd.read_csv("titanic.csv")
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
635,636,1,2,"Davis, Miss. Mary",female,28.0,0,0,237668,13.0000,NaN,S
836,837,0,3,"Pasic, Mr. Jakob",male,21.0,0,0,315097,8.6625,NaN,S
815,816,0,1,"Fry, Mr. Richard",male,NaN,0,0,112058,0.0000,B102,S
136,137,1,1,"Newsom, Miss. Helen Monypeny",female,19.0,0,2,11752,26.2833,D47,S
406,407,0,3,"Widegren, Mr. Carl/Charles Peter",male,51.0,0,0,347064,7.7500,NaN,S


In [24]:
df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis = 1, inplace = True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [25]:
df.shape

(891, 8)

In [26]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

# Split the Dataset into Features and Target 

In [27]:
X = df.drop("Survived", axis = 1)
y = df["Survived"]

In [28]:
X.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C


In [29]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

# Split the Dataset into Training and Testing Data

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [31]:
print(X.shape, X_train.shape, X_test.shape)

(891, 7) (712, 7) (179, 7)


# Preprocessing Without Column Transformer

### Imputation

In [32]:
# Load the SimpleImputer Function
age_si = SimpleImputer()
embarked_si = SimpleImputer(strategy = "most_frequent")

# Training Data
X_train_age = age_si.fit_transform(X_train[["Age"]])
X_train_embarked = embarked_si.fit_transform(X_train[["Embarked"]])

# Testing Data
X_test_age = age_si.transform(X_test[["Age"]])
X_test_embarked = embarked_si.transform(X_test[["Embarked"]])

In [33]:
print(X_train_age.shape, X_test_embarked.shape)

(712, 1) (179, 1)


### OneHotEncoding

In [34]:
# Load the OneHotEncoder Function
ohe_sex = OneHotEncoder(sparse_output = False, handle_unknown = "ignore")
ohe_embarked = OneHotEncoder(sparse_output = False, handle_unknown = "ignore")

# Training Data
X_train_sex = ohe_sex.fit_transform(X_train[["Sex"]])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

# Testing Data
X_test_sex = ohe_sex.transform(X_test[["Sex"]])
X_test_embarked = ohe_embarked.transform(X_test_embarked)

In [35]:
print(X_train_sex.shape, X_train_embarked.shape)

(712, 2) (712, 3)


In [36]:
X_train_embarked

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

# Concatenation

In [37]:
X_train_rem = X_train.drop(["Sex", "Age", "Embarked"], axis = 1)
X_test_rem = X_test.drop(["Sex", "Age", "Embarked"], axis = 1)

In [38]:
X_train_transformed = np.concatenate((X_train_rem, X_train_age, X_train_sex, X_train_embarked), axis = 1)
X_test_transformed = np.concatenate((X_test_rem, X_test_age, X_test_sex, X_test_embarked), axis = 1)

In [39]:
print(X_train_transformed.shape, X_test_transformed.shape)

(712, 10) (179, 10)


# Train the Model

In [40]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


# Prediction

In [41]:
y_pred = clf.predict(X_test_transformed)
y_pred

array([0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1], dtype=int64)

# Model Evaluation

In [42]:
accuracy_score(y_pred, y_test)

0.7821229050279329

In [46]:
import pickle
import os

In [45]:
os.makedirs("models", exist_ok = True)

pickle.dump(ohe_sex, open("models/ohe_sex.pkl", "wb"))
pickle.dump(ohe_embarked, open("models/ohe_embarked.pkl", "wb"))
pickle.dump(clf, open("models/clf.pkl", "wb"))